In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


<img src='../../../common/logo_DH.png' align='left' width=35%/>

## Desafío: Clasificación de artículos de diario

Trabajaremos con un dataset de noticias de los diarios Clarin y Pagina12. El objetivo de la práctica será implementar un modelo que permita predecir de qué diario proviene una noticia.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,confusion_matrix
import unidecode
from nltk.corpus import stopwords 


### 1. Importamos los datos

* Importen los datos con pandas y generen un dataframe agregando una columna 'clase' que indique si son noticias de Clarin o de Pagina12.

Las noticias de Clarin se encuentran en '../Data/clarin.csv' y las de Pagina12 en '../Data/pagina12.csv'.

* Concatenen ambos data sets en un solo dataframe.

* ¿Cuántas noticias tenemos de cada diario?

* ¿Qué columnas tiene el dataframe? 

In [ ]:
df_clarin = pd.read_csv('../Data/clarin.csv')
df_clarin['class'] = 0

In [ ]:
df_p12 = pd.read_csv('../Data/pagina12.csv')
df_p12['class'] = 1

In [ ]:
df = pd.concat([df_clarin,df_p12])

In [ ]:
df.columns

In [ ]:
print('Noticias de clarin:',(df['class']==0).sum())
print('Noticias de pagina:',(df['class']==1).sum())

### 2. Limpieza

#### 2.1 Faltantes

A partir del dataset observamos que los campos que probablemente contengan el vocabulario relevante son "cuerpo", "título" y "resumen".

* Saquen del análisis los registros que no tienen cuerpo o título disponible

* Completen los resúmenes faltantes con una campo en blanco


In [ ]:
df.dropna(subset=['cuerpo','titulo'],inplace=True)

In [ ]:
df.shape

In [ ]:
df.loc[df['resumen'].isnull(),['resumen']]='';

In [ ]:
df['resumen'].isnull().any()

#### 2.2 Suplementos relevantes

Para mejorar la clasificación es conveniente retirar las secciones donde los dos diarios utilizan un vocabulario similar y muy específico del dominio como, por ejemplo, las relacionadas a deportes.

* Miren las secciones dentro de la columna 'suplemento': Ojo que hay secciones de deportes con diferente nombre por ejemplo '/deportes/futbol/'

* Remuevan las noticias de deportes

In [ ]:
deportes=df['suplemento'].apply(lambda x: 'deporte' in str(x).lower());
df.drop(index=df[deportes].index,inplace=True)

df.drop(index=df[df['suplemento']=='/br/'].index,inplace=True) # Tiro articulos en portugues

df.shape
#espectaculos=df['suplemento'].apply(lambda x: 'espectaculos' in str(x).lower());
#df.drop(index=df[espectaculos].index,inplace=True)

#### 2.3 Corpus

El data set tiene informacion relevante en las columnas 'título', 'resumen' y 'cuerpo', de modo que podemos generar una nueva columna que sea la concatenación de estas tres. 

* Generen dicha columna, que será nuestro corpus de documentos.


In [ ]:
df['noticia']=df['titulo']+' '+df['resumen']+' '+df['cuerpo'];

In [ ]:
df['noticia']

## 3. Modelo

### 3.1 

* Vectoricen el corpus de textos resultante con CountVectorizer, removiendo stopwords. Usen el argumento strip_accents='unicode' para remover tildes del texto.

Atención: las stopwords importadas de nltk contienen tildes. Elimínenlas antes de vectorizar el corpus.

* ¿Cuál es la dimensión de la matriz de features?

* Apliquen un modelo Naive Bayes con un split simple entre train y test. 

* ¿Cuál es el accuracy obtenido?  

* Dibujen la matriz de confusión.

In [ ]:
# Excluimos stopwords
stop_words = stopwords.words('spanish');
stop_words=[unidecode.unidecode(word.lower()) for word in stop_words ]; # quitamos acentos

Train,Test=train_test_split(df[['noticia','class']],stratify=df['class'],random_state=3);

Train.reset_index(drop=True,inplace=True);
Test.reset_index(drop=True,inplace=True);

vectorizer=CountVectorizer(strip_accents='unicode',stop_words=stop_words);
vectorizer.fit(Train['noticia']);

X_train=vectorizer.transform(Train['noticia']);
X_test=vectorizer.transform(Test['noticia']);

y_train=Train['class'];
y_test=Test['class'];

NBC=MultinomialNB();

NBC.fit(X_train.todense(),Train['class']);

test_pred=NBC.predict(X_test.todense());

print('Training set shape:',X_train.shape)

print('\nTest Accuracy:',accuracy_score(Test['class'],test_pred))

print('\nConfusion Matrix:\n',confusion_matrix(Test['class'],test_pred))




### 3.2 Optimización del modelo

* Hagan una gridsearch cross validation variando el hiperparámetro alpha en el rango (0;0.1)

* Vean la accuracy y la matriz de confusión obtenida con el mejor modelo, en el test set.

In [ ]:
len(np.arange(0.1,2,0.1))

In [ ]:
skf=StratifiedKFold(n_splits=3,random_state=0,shuffle=True);

params={'alpha':np.arange(0.1,2,0.1)};
GS_CV=GridSearchCV(MultinomialNB(),params,cv=skf,verbose=1,n_jobs=-1);
GS_CV.fit(X_train,y_train);
print('best score:',GS_CV.best_score_)
print('best params:',GS_CV.best_params_)

best_model=GS_CV.best_estimator_;
best_model.fit(X_train,y_train); # entrenamos en todo el training set

print('\nTest set:\n')

test_pred=best_model.predict(X_test);

print('accuracy:',accuracy_score(Test['class'],test_pred))

print('\nconfusion:\n',confusion_matrix(Test['class'],test_pred))


### 4. Análisis de los resultados 

El modelo entrenado tiene el atributo "feature_log_prob" que contiene el logaritmo de los coeficientes $\theta_{yi}$, que representan la probabilidad de que el término i-ésimo pertenezca a la clase $y$.

¿Cuáles son las features (palabras) que mejor separan a las dos clases?

* Calculen el cociente entre los logaritmos de los coeficientes estimados para la clase "clarin" y para "pagina12". ¿Cuáles términos mustran mayor diferencia entre ambos valores?

In [ ]:
relative_importance=best_model.feature_log_prob_[0]/best_model.feature_log_prob_[1];
# Los valores son log-prob (negativos) de modo que relative_importance < 1 implica que
# el coeficiente asignado en la clase 0 (clarin) es mayor que en la clase 1 (pagina) y viceveras

sns.distplot(relative_importance)
plt.xlabel('Importancia Relativa')

features=np.array(vectorizer.get_feature_names());

indices=np.argsort(relative_importance);

print('Términos representativos de Clarín:\n')
print(features[indices[:100]])

print('\nTérminos representativos de Página12:\n')
print(features[indices[-100:]])
